# Thrones2Vec

Using only the raw text of [A Song of Ice and Fire](https://en.wikipedia.org/wiki/A_Song_of_Ice_and_Fire), we'll derive and explore the semantic properties of its words.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re

In [3]:
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Set up logging**

In [5]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

**Download NLTK tokenizer models (only the first time)**

In [6]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/davebrunner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/davebrunner/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Prepare Corpus

**Load books from files**

In [7]:
book_filenames = sorted(glob.glob("./data/*.txt"))

In [8]:
print("Found books:")
book_filenames

Found books:


['./data/got1.txt',
 './data/got2.txt',
 './data/got3.txt',
 './data/got4.txt',
 './data/got5.txt']

**Combine the books into one string**

In [9]:
corpus_raw = u""
for book_filename in book_filenames:
    print("Reading '{0}'...".format(book_filename))
    with codecs.open(book_filename, "r", "utf-8") as book_file:
        corpus_raw += book_file.read()
    print("Corpus is now {0} characters long".format(len(corpus_raw)))
    print()

Reading './data/got1.txt'...
Corpus is now 1770659 characters long

Reading './data/got2.txt'...
Corpus is now 4071041 characters long

Reading './data/got3.txt'...
Corpus is now 6391405 characters long

Reading './data/got4.txt'...
Corpus is now 8107945 characters long

Reading './data/got5.txt'...
Corpus is now 9719485 characters long



**Split the corpus into sentences**

In [10]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [11]:
raw_sentences = tokenizer.tokenize(corpus_raw)

In [12]:
#convert into a list of words
#rtemove unnnecessary,, split into words, no hyphens
#list of words
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [13]:
#sentence where each word is tokenized
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [14]:
print(raw_sentences[5])
print(sentence_to_wordlist(raw_sentences[5]))

Heraldic crest by Virginia Norey.
['Heraldic', 'crest', 'by', 'Virginia', 'Norey']


In [15]:
token_count = sum([len(sentence) for sentence in sentences])
print("The book corpus contains {0:,} tokens".format(token_count))

The book corpus contains 1,818,103 tokens


## Train Word2Vec

In [16]:
#ONCE we have vectors
#step 3 - build model
#3 main tasks that vectors help with
#DISTANCE, SIMILARITY, RANKING

# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 300
# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

# Number of iterations theneural network is trained
epochs = 5

In [17]:
thrones2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    vector_size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling,
    epochs= epochs
)

2021-12-29 11:47:43,913 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.025)', 'datetime': '2021-12-29T11:47:43.913634', 'gensim': '4.0.1', 'python': '3.8.12 (default, Oct 12 2021, 06:23:56) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}


In [18]:
thrones2vec.build_vocab(sentences)

2021-12-29 11:47:43,952 : INFO : collecting all words and their counts
2021-12-29 11:47:43,955 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-12-29 11:47:44,000 : INFO : PROGRESS: at sentence #10000, processed 140984 words, keeping 10280 word types
2021-12-29 11:47:44,050 : INFO : PROGRESS: at sentence #20000, processed 279730 words, keeping 13558 word types
2021-12-29 11:47:44,109 : INFO : PROGRESS: at sentence #30000, processed 420336 words, keeping 16598 word types
2021-12-29 11:47:44,160 : INFO : PROGRESS: at sentence #40000, processed 556581 words, keeping 18324 word types
2021-12-29 11:47:44,216 : INFO : PROGRESS: at sentence #50000, processed 686247 words, keeping 19714 word types
2021-12-29 11:47:44,282 : INFO : PROGRESS: at sentence #60000, processed 828497 words, keeping 21672 word types
2021-12-29 11:47:44,344 : INFO : PROGRESS: at sentence #70000, processed 973830 words, keeping 23093 word types
2021-12-29 11:47:44,429 : INFO : PROGRESS: at 

In [19]:
print("Word2Vec vocabulary length:", len(thrones2vec.wv))

Word2Vec vocabulary length: 17277


**Start training, this might take a minute or two...**

In [28]:
thrones2vec.train(sentences,total_words=(len(thrones2vec.wv)), epochs=80)

2021-12-29 11:50:38,104 : WARNING : Effective 'alpha' higher than previous training cycles
2021-12-29 11:50:38,106 : INFO : Word2Vec lifecycle event {'msg': 'training model with 8 workers on 17277 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=7', 'datetime': '2021-12-29T11:50:38.106689', 'gensim': '4.0.1', 'python': '3.8.12 (default, Oct 12 2021, 06:23:56) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'train'}
2021-12-29 11:50:39,132 : INFO : EPOCH 1 - PROGRESS: at 2021.53% words, 267460 words/s, in_qsize 16, out_qsize 0
2021-12-29 11:50:40,145 : INFO : EPOCH 1 - PROGRESS: at 4680.60% words, 309237 words/s, in_qsize 15, out_qsize 0
2021-12-29 11:50:41,187 : INFO : EPOCH 1 - PROGRESS: at 7454.91% words, 325155 words/s, in_qsize 15, out_qsize 0
2021-12-29 11:50:42,167 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-12-29 11:50:42,174 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-12

(112357418, 145448240)

**Save to file, can be useful later**

In [29]:
if not os.path.exists("trained"):
    os.makedirs("trained")

In [30]:
thrones2vec.save(os.path.join("trained", "thrones2vec.w2v"))

2021-12-29 11:56:22,249 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'trained/thrones2vec.w2v', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-12-29T11:56:22.249409', 'gensim': '4.0.1', 'python': '3.8.12 (default, Oct 12 2021, 06:23:56) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'saving'}
2021-12-29 11:56:22,250 : INFO : not storing attribute cum_table
2021-12-29 11:56:22,357 : INFO : saved trained/thrones2vec.w2v


## Explore the trained model.

In [31]:
thrones2vec = w2v.Word2Vec.load(os.path.join("trained", "thrones2vec.w2v"))

2021-12-29 11:56:22,378 : INFO : loading Word2Vec object from trained/thrones2vec.w2v
2021-12-29 11:56:22,407 : INFO : loading wv recursively from trained/thrones2vec.w2v.wv.* with mmap=None
2021-12-29 11:56:22,408 : INFO : setting ignored attribute cum_table to None
2021-12-29 11:56:22,639 : INFO : Word2Vec lifecycle event {'fname': 'trained/thrones2vec.w2v', 'datetime': '2021-12-29T11:56:22.639422', 'gensim': '4.0.1', 'python': '3.8.12 (default, Oct 12 2021, 06:23:56) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'loaded'}


### Compress the word vectors into 2D space and plot them

In [32]:
#my video - how to visualize a dataset easily
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0)

In [33]:
all_word_vectors_matrix = thrones2vec.wv.vectors

**Train t-SNE, this could take a minute or two...**

In [34]:
all_word_vectors_matrix_2d = tsne.fit_transform(all_word_vectors_matrix)

/Users/davebrunner/opt/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/davebrunner/opt/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


**Plot the big picture**

In [35]:
#plot point in 2d space
points = pd.DataFrame(
    [
        (word, coords[0], coords[1])
        for word, coords in [
            (word, all_word_vectors_matrix_2d[thrones2vec.wv[word].index])
            for word in thrones2vec.wv
        ]
    ],
    columns=["word", "x", "y"]
)

KeyError: "Key '0.08391094952821732' not present"

In [ ]:
points.head(10)

NameError: name 'points' is not defined

In [ ]:
sns.set_context("poster")

In [ ]:
points.plot.scatter("x", "y", s=10, figsize=(20, 12))

NameError: name 'points' is not defined

**Zoom in to some interesting places**

In [ ]:
def plot_region(x_bounds, y_bounds):
    slice = points[
        (x_bounds[0] <= points.x) &
        (points.x <= x_bounds[1]) & 
        (y_bounds[0] <= points.y) &
        (points.y <= y_bounds[1])
    ]
    
    ax = slice.plot.scatter("x", "y", s=35, figsize=(10, 8))
    for i, point in slice.iterrows():
        ax.text(point.x + 0.005, point.y + 0.005, point.word, fontsize=11)

**People related to Kingsguard ended up together**

In [ ]:
plot_region(x_bounds=(4.0, 4.2), y_bounds=(-0.5, -0.1))

NameError: name 'plot_region' is not defined

**Food products are grouped nicely as well. Aerys (The Mad King) being close to "roasted" also looks sadly correct**

In [ ]:
plot_region(x_bounds=(0, 1), y_bounds=(4, 4.5))

NameError: name 'plot_region' is not defined

### Explore semantic similarities between book characters

**Words closest to the given word**

In [36]:
thrones2vec.wv.most_similar("Stark")

[('Eddard', 0.5477069020271301),
 ('Ned', 0.49601152539253235),
 ('interred', 0.40962404012680054),
 ('dated', 0.39974522590637207),
 ('Snowbeard', 0.3994207978248596),
 ('Gwynesse', 0.3989529311656952),
 ('Provided', 0.39573848247528076),
 ('Knelt', 0.3955308794975281),
 ('fishwife', 0.3922034800052643),
 ('Harlon', 0.39078402519226074)]

In [37]:
thrones2vec.wv.most_similar("Aerys")

[('II', 0.5436996817588806),
 ('Targaryen', 0.45346009731292725),
 ('Conciliator', 0.4516374170780182),
 ('LYANNA', 0.43061625957489014),
 ('BENJEN', 0.42489373683929443),
 ('V', 0.4218860864639282),
 ('inseparable', 0.4204847514629364),
 ('VISERYS', 0.41132625937461853),
 ('lusted', 0.4111787974834442),
 ('Bells', 0.40994927287101746)]

In [38]:
thrones2vec.wv.most_similar("direwolf")

[('lifelike', 0.4667336642742157),
 ('GHOST', 0.4485681653022766),
 ('nuzzle', 0.43722081184387207),
 ('SHAGGYDOG', 0.43450573086738586),
 ('RICKON', 0.42921459674835205),
 ('cornered', 0.4095626175403595),
 ('BRANDON', 0.4061349034309387),
 ('crunching', 0.403515487909317),
 ('WIND', 0.4022693336009979),
 ('bounding', 0.3885434567928314)]

**Linear relationships between word pairs**

In [39]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = thrones2vec.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{start1} is related to {end1}, as {start2} is related to {end2}".format(**locals()))
    return start2

In [40]:
nearest_similarity_cosmul("Stark", "Winterfell", "Riverrun")
nearest_similarity_cosmul("Jaime", "sword", "wine")
nearest_similarity_cosmul("Arya", "Nymeria", "dragons")

Stark is related to Winterfell, as Tully is related to Riverrun
Jaime is related to sword, as flagon is related to wine
Arya is related to Nymeria, as buys is related to dragons


'buys'

In [41]:
#1 Print the one-hot-vector for Arryn
thrones2vec.wv["Arry"]

array([ 4.82532173e-01,  4.98011380e-01, -3.06800902e-01,  1.67669371e-01,
        1.56866759e-01,  1.87117094e-03, -3.25347394e-01,  3.51193160e-01,
       -6.24717712e-01,  4.88662988e-01,  1.70161575e-01,  2.58375198e-01,
       -3.76088977e-01, -4.96356152e-02,  7.18446553e-01, -7.84900844e-01,
       -1.03377116e+00, -4.12113905e-01,  4.34832543e-01, -4.31748599e-01,
       -4.56624061e-01, -2.37316206e-01,  3.39304805e-02, -4.78552401e-01,
        5.20434141e-01,  1.50782943e-01, -1.26956999e-01, -7.80486837e-02,
        1.10623218e-01, -6.71653375e-02,  4.67048325e-02,  3.42068300e-02,
       -8.91234726e-02, -9.25141126e-02,  1.95042312e-01, -3.44330549e-01,
        7.77295589e-01,  5.38673103e-01, -4.62925397e-02,  4.89073753e-01,
        4.35944289e-01,  1.65313914e-01, -2.10371420e-01,  5.67713976e-01,
       -4.43167448e-01,  1.62512153e-01, -7.12837502e-02, -1.72893539e-01,
       -7.90975839e-02,  4.95149195e-01, -1.15439609e-01,  3.44232023e-02,
       -1.23586357e-01,  

In [42]:
#2
print("There are:", len(thrones2vec.wv),"Vetors")

There are: 17277 Vetors


In [43]:
#3
number_of_similarities = 7
word = "Lannister"
print("The {} most similar words for {} are:".format(number_of_similarities,word), thrones2vec.wv.most_similar(word,topn=number_of_similarities))

The 7 most similar words for Lannister are: [('claimant', 0.40874043107032776), ('crimson', 0.3956228196620941), ('Tywin', 0.3758547604084015), ('Genna', 0.3757480978965759), ('flatly', 0.3663230836391449), ('Kingslayer', 0.36607250571250916), ('someday', 0.3568412959575653)]


In [44]:
#4
word1 = "Jon"
word2 = "Ygritte"
print("Similarity of {} and {}".format(word1,word2,), thrones2vec.wv.similarity(word1, word2))

Similarity of Jon and Ygritte 0.41419047


In [45]:
#5
sentence1 = ["Hodor", "that", "was", "all", "he", "ever", "said"]
sentence2 = ["Hold", "the", "door"]
sum_of_similarities = 0

for x in sentence1:
    for y in sentence2:
        sum_of_similarities += thrones2vec.wv.similarity(x,y)

print(sum_of_similarities/(len(sentence1)*len(sentence2)))
print(sentence1,sentence2)

word1 = "Jon"
word2 = "Snow"
print("Similarity of {} and {}".format(word1,word2,), thrones2vec.wv.similarity(word1, word2))

0.19486017170406522
['Hodor', 'that', 'was', 'all', 'he', 'ever', 'said'] ['Hold', 'the', 'door']
Similarity of Jon and Snow 0.6545374
